In [1]:
import numpy as np
from os import listdir
from os.path import isfile, join
import SimpleITK as sitk
from MyFunctions.LoadImages import LoadImages

# Mask PET with CT

In [ ]:
pathCT_Msk = "../../../Masters_Repo/TrainingData/PET/Process/CT_Msk/"
pathPET = "../../../Masters_Repo/TrainingData/PET/Process/PET_Img/"

def getFiles(path):
    return [f for f in listdir(path) if isfile(join(path,f))]

files_CT_Msk = getFiles(pathCT_Msk)
files_PET = getFiles(pathPET)

In [ ]:

def SetMetadata(Scan_Metadata,IMAGE):
        """Sets a given IMAGE with a given METADATA"""
        IMAGE.SetSpacing(Scan_Metadata[0])
        IMAGE.SetOrigin(Scan_Metadata[1])
        IMAGE.SetDirection(Scan_Metadata[2])

def maskPET(PETfilenames, CTfilenames , PETdirPath, CTdirPath):
    if(np.size(PETfilenames) == np.size(CTfilenames)):
        for i in range(np.size(PETfilenames)):
            curr_PET = PETfilenames[i]
            curr_CT = CTfilenames[i]
            PET_file,_,PET_Meta = LoadImages(ScanType="PET", ScanClass="Image",
                       ImgPath=PETdirPath ,ScanName=curr_PET, n_Scans=1, ImgSize=0, ImgDepth=0, Orientation="Axial").LoadScan()
            CT_msk_file,_,_ = LoadImages(ScanType="CT", ScanClass="Mask",
                       MskPath=CTdirPath,ScanName=curr_CT, n_Scans=1, ImgSize=0, ImgDepth=0, Orientation="Axial").LoadScan()
            PET_Msked = PET_file*CT_msk_file
            Model_Out = sitk.GetImageFromArray(PET_Msked)
            SetMetadata(PET_Meta, Model_Out)
            WritePath = join('../../../Masters_Repo/TrainingData/PET/Process/PET_Msk_Uncorrected', curr_PET)
            sitk.WriteImage(Model_Out,WritePath)
    else:
        return print("Error: Differering number of PET and CT masks. Cannot mask!")
maskPET(files_PET,files_CT_Msk, pathPET ,pathCT_Msk)
print("Done! Check if masking process was correct..")

# Subtract CT Msk with PET msk

In [3]:
pathCT_Msk = "../../../Masters_Repo/TrainingData/PET/Process/CT_Msk/"
pathPET_Msk = "../../../Masters_Repo/TrainingData/PET/Process/PET_Msk_Corrected/"

def getFiles(path):
    return [f for f in listdir(path) if isfile(join(path,f))]

files_CT_Msk = getFiles(pathCT_Msk)
files_PET_Msk = getFiles(pathPET_Msk)

In [8]:
def SetMetadata(Scan_Metadata,IMAGE):
        """Sets a given IMAGE with a given METADATA"""
        IMAGE.SetSpacing(Scan_Metadata[0])
        IMAGE.SetOrigin(Scan_Metadata[1])
        IMAGE.SetDirection(Scan_Metadata[2])

def maskPETCT(PETfilenames, CTfilenames , PETdirPath, CTdirPath):
    if(np.size(PETfilenames) == np.size(CTfilenames)):
        for i in range(np.size(PETfilenames)):
            curr_PET = PETfilenames[i]
            curr_CT = CTfilenames[i]
            PET_file,_,PET_Meta = LoadImages(ScanType="PET", ScanClass="Mask",
                       MskPath=PETdirPath ,ScanName=curr_PET, n_Scans=1, ImgSize=0, ImgDepth=0, Orientation="Axial").LoadScan()
            CT_msk_file,_,_ = LoadImages(ScanType="CT", ScanClass="Mask",
                       MskPath=CTdirPath,ScanName=curr_CT, n_Scans=1, ImgSize=0, ImgDepth=0, Orientation="Axial").LoadScan()
            PET_Msked = CT_msk_file-PET_file
            #Added this line due to -1 sometimes poping up. Probably slight mask error
            PET_Msked[PET_Msked < 0] = 0
            Model_Out = sitk.GetImageFromArray(PET_Msked)
            SetMetadata(PET_Meta, Model_Out)
            WritePath = join('../../../Masters_Repo/TrainingData/PET/Process/Subtracted_Msks', curr_PET)
            sitk.WriteImage(Model_Out,WritePath)
    else:
        return print("Error: Differering number of PET and CT masks. Cannot mask!")

In [9]:
maskPETCT(files_PET_Msk,files_CT_Msk, pathPET_Msk ,pathCT_Msk)
print("Done! Check if masking process was correct..")

Done! Check if masking process was correct..
